In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import zipfile
from collections import defaultdict
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

In [2]:
# read csv for valid row_cols for samples to draw. They are based on the share of agriculture (HR Landcover maps) within a S3 pixel
thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')

/tmp/ipykernel_2988869/2653007668.py:2: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')


In [3]:
for col in thresh_csv.columns:
    print(f'finding indices for {col}')
    nested = [entry.split('_') for entry in thresh_csv[col] if type(entry) == str]
    rows, cols = zip(*nested)
    print(len(rows))
# only use Thresh50, as all valid row_col will be here. For other THresh sets, a subset from this can be taken

finding indices for Thresh100
4941
finding indices for Thresh90
32250
finding indices for Thresh80
60666
finding indices for Thresh70
92847
finding indices for Thresh60
127768
finding indices for Thresh50
165877


In [4]:
year = 2018
force_path = '/data/Aldhani/eoagritwin/force/output/S3/2018/X0073_Y0046/TSI/'
files = getFilelist(force_path, '.tif')
files_annual = [file for file in files if str(year) in file]
# delete all other files
[RasterKiller(file) for file in files if str(year) not in file.split('_')[-1]]
indices = list(set([file_annual.split('_')[-3] for file_annual in files_annual]))

In [ ]:
# warp em
#for index in indices:
index = indices[3]
index_files = [file_annual for file_annual in files_annual if index in file_annual]
aa = warp_to_template(index_files[0], 
                      '/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
                      #mask_path='/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
                      outPath='/data/Aldhani/eoagritwin/et/Auxiliary/trash/uuint_nomask.tif',
                      outType=gdal.GDT_Int16)

In [25]:
index_files[3]

'/data/Aldhani/eoagritwin/force/output/S3/2018/X0073_Y0046/TSI/20171001-20190228_001-365_HL_TSA_SEN2L_EVI_TSI_20180104.tif'

In [23]:
q = gdal.Open('/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif')
qq = q.GetRasterBand(1).ReadAsArray()
np.unique(qq)


array([0, 1], dtype=int16)

In [24]:
aa = gdal.Open(index_files[3])
bb = aa.GetRasterBand(1).ReadAsArray()
bb

array([[3188, 3048, 3428, ..., 1514, 1201, 1173],
       [3498, 3528, 3568, ..., 1457, 1767, 1746],
       [3649, 3254, 3630, ..., 1384, 1190, 1641],
       ...,
       [3684, 3692, 3674, ..., 2132, 2099, 2199],
       [3698, 3701, 3645, ..., 2119, 2109, 2242],
       [3425, 3463, 3681, ..., 2085, 2075, 2269]],
      shape=(3000, 3000), dtype=int16)

In [ ]:
files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Sentinel3/raw', '.nc'))
aa = getDataFromNC(files[0])
bb = getAllDatesS3(files[0])
year = 2020

In [38]:
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

# create a maks for germany
mask = makeGermanyMaskforNC('/data/Aldhani/eoagritwin/misc/gadm41_DEU_shp/gadm41_DEU_0.shp', yearFiles[0])

# make dictionary that stores .tif names and related accDates to easier search to close observations with S2 data
lookUp = {}

# set storPath for exported tiffs
storPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/'
LST_path = f'{storPath}LST/daily_observations_all/{year}/'
monthly_composites_path = f'{storPath}LST/monthly_composites/{year}/'
os.makedirs(LST_path, exist_ok=True)
os.makedirs(monthly_composites_path, exist_ok=True)
yearCont = []
file = yearFiles[0]


In [40]:
# loop over files and export to .tif at location storPath
#for i, file in enumerate(yearFiles):
print(file)
accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
#     convertNCtoTIF(file, LST_path, file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True)

# write filenames and corresponding dates into dictionary for easier datesearch later
lookUp[file.split('.nc')[0]] = accDateTimes

dat = getDataFromNC(file)
monthCont = []
df = pd.Series(accDateTimes)
counts_per_day = df.dt.floor("D").value_counts().sort_index()
# vectors for indexing over days
cumulative_day_counts_end = np.asarray(np.cumsum(counts_per_day))
cumulative_day_counts_start = np.insert(cumulative_day_counts_end, 0 ,0)

# cumulative_day_counts_start = np.array(cumulative_day_counts_start)
# cumulative_day_counts_end = np.array(cumulative_day_counts_end)

# aggreagate by median
# stack_list = [
#     np.nanmedian(dat[:, :, start:end], axis=2)
#     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
# ] 
# fin_block = np.dstack(stack_list)

# MM = int_to_Month[file.rsplit('-', maxsplit=1)[-1].split('.')[0]]
# bands = [f'{MM}_Day_{b+1}' for b in range(fin_block.shape[2])]
# fin_block = fin_block * mask[:, :, np.newaxis]
# fin_block[fin_block == 0] = np.nan
# exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_mean.tif', bands, fin_block)

# # aggreagate by min
# stack_list = [
#     np.nanmin(dat[:, :, start:end], axis=2)
#     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
# ] 
# fin_block = np.dstack(stack_list)
# fin_block = fin_block * mask[:, :, np.newaxis]
# fin_block[fin_block == 0] = np.nan
# exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_min.tif', bands, fin_block)

# # aggreagate by max
# stack_list = [
#     np.nanmax(dat[:, :, start:end], axis=2)
#     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
# ] 
# fin_block = np.dstack(stack_list)
# fin_block = fin_block * mask[:, :, np.newaxis]
# fin_block[fin_block == 0] = np.nan
# exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_max.tif', bands, fin_block)


/data/Aldhani/eoagritwin/et/Sentinel3/raw/Germany_2020-01.nc


In [ ]:
str(counts_per_day.index[0].date())

'2020-01-01'

In [58]:
file

'/data/Aldhani/eoagritwin/et/Sentinel3/raw/Germany_2020-01.nc'